# Notebook Imports


In [1]:
import pandas as pd
import numpy as np

# Constants

In [123]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'
VOCAB_SIZE = 2500


TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'


# Read and Load Features from .txt Files into Numpy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 258360
Nr of rows in test file 117776


In [7]:
print('Nr of emails in training file', np.unique(sparse_train_data[:, 0]).size) # all rows in first column

Nr of emails in training file 4013


In [8]:
print('Nr of emails in test file', np.unique(sparse_test_data[:, 0]).size) # all rows in first column

Nr of emails in test file 1724


### How to create an Empty DataFrame

In [14]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [10]:
len(column_names)

2502

In [11]:
index_names = np.unique(sparse_test_data[:, 0])
index_names

array([   8,   12,   14, ..., 5788, 5792, 5793])

In [18]:
# Creates a data frame with NaN values
full_train_data = pd.DataFrame(index=index_names, columns=column_names) 

# Fills the NaN values with value of choice
full_train_data.fillna(value=0, inplace=True)

C:\Users\tanic\AppData\Local\Temp\ipykernel_22676\1828918322.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_train_data.fillna(value=0, inplace=True)


In [19]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from a Sparse Matrix

array([[ 0, 31,  1,  1],
       [ 0, 33,  1,  1],
       [ 0, 44,  1,  1]])

In [21]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
        """
        Form a full matrix from a sparse matrix. Return a pandas dataframe. 
        Keyword arguments:
        sparse_matrix -- numpy array
        nr_words -- size of the vocabulary. Total number of tokens. 
        doc_idx -- position of the document id in the sparse matrix. Default: 1st column
        word_idx -- position of the word id in the sparse matrix. Default: 2nd column
        cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
        freq_idx -- position of occurence of word in sparse matrix. Default: 4th column

        """
        column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
        doc_id_names = np.unique(sparse_matrix[:, 0])

        # Creates a data frame with NaN values
        full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names) 

        # Fills the NaN values with value of choice
        full_matrix.fillna(value=0, inplace=True)

        # Grabs data from sparse matrix and adds it into the full matrix
        for i in range(sparse_matrix.shape[0]):
                # These correspond to the columns of the dataframe
                doc_nr = sparse_matrix[i][doc_idx]
                word_id = sparse_matrix[i][word_idx]
                label = sparse_matrix[i][cat_idx]    
                occurrence = sparse_matrix[i][freq_idx]           

                # Selects particular cell in the dataframe that was created
                full_matrix.at[doc_nr, 'DOC_ID']= doc_nr
                full_matrix.at[doc_nr, 'CATEGORY']= label
                full_matrix.at[doc_nr, word_id]= occurrence

        full_matrix.set_index('DOC_ID', inplace=True)
        return full_matrix
                

In [22]:
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

C:\Users\tanic\AppData\Local\Temp\ipykernel_22676\3471363982.py:20: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace=True)


In [23]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model
## Calculating the Probability of Spam 

In [40]:
# Challenge: Calculate the probability of spam - the percent of spam messages
# in the training dataset. 
# Store this value in a variable called prob_spam. 

# My solution
# prob_spam = len(full_train_data[full_train_data.CATEGORY == 1]) / len(full_train_data)
# prob_spam

# Lesson solution
# full_train_data.CATEGORY.size #total emails
# full_train_data.CATEGORY.sum() #number of spam emails
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

Probability of spam is 0.310989284824321


## Total Number of Words/Tokens

In [42]:
# Picks all columns except CATEGORY
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY'] 
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
# Counts the number of tokens per email (Number of words)
email_lengths = full_train_features.sum(axis=1) #Sum of columns
# email_lengths.shape
# email_lengths[:5]

total_wc = email_lengths.sum()
total_wc

429241

## Nmber of Tokens in Spam & Ham Emails
**Challenge:** Create a subset of the email_lengths series that only contains the spam messages. Call the subset spam_lengths.
Then count the total number of words that occur in spam emails. 

Do the same for the non-spam emails. Create a subset called ham_lengths. Then count the total number of words that occur in the ham emails. 

In [81]:
# Lesson solution
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape
spam_wc = spam_lengths.sum()

ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape
ham_wc = ham_lengths.sum()

print(spam_wc)
print(ham_wc)

176334
252907


In [82]:
# Subset check must equal 0
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [84]:
nonspam_wc = ham_lengths.sum()

252907

Challenge: Can you figure out if spam emails or non-spam emails tend to be longer? 
Which category contains more tokens? Take a guess and the verify using the dataset

In [89]:
print('Average nr of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.3f}'.format(ham_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 141
Average nr of words in ham emails 91.467


## Summing the Tokens Occuring in Spam

In [90]:
full_train_features.shape

(4013, 2500)

In [97]:
train_spam_tokens = full_train_features[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
train_spam_tokens.shape

(1248, 2500)

In [101]:
# Sum of tokens columns going veritcally
# Apply Laplace Smoothing
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 # +1 added to account for 0s
summed_spam_tokens.shape

(2500,)

In [102]:
summed_spam_tokens.tail()

2495     3
2496     2
2497    17
2498    15
2499     2
dtype: int64

In [ ]:
# Challenge: Repeat this process for the ham messages. Sum the tokens that occur
# in the non-spam messages. Store the values in a variable called summed_ham_tokens


## Summing the Tokens Occurring in Ham

In [104]:
# Sum of tokens columns going veritcally
train_ham_tokens = full_train_features[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [105]:
summed_ham_tokens.shape

(2500,)

In [107]:
summed_ham_tokens.tail()

2495    27
2496    35
2497    20
2498    15
2499    28
dtype: int64

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [109]:
# Remember to reverse Laplas smoothing
# Iterates on each row

prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012184
1    0.005228
2    0.006805
3    0.011307
4    0.006816
dtype: float64

In [110]:
# The probabilities above must sum to 1. 
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [113]:
prob_tokens_nonspam = summed_ham_tokens / (ham_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0

# P(Token) - Probability that Token Occurs

In [118]:
prob_tokens_all  = full_train_features.sum(axis=0) / total_wc #Laplas not necessary

In [119]:
prob_tokens_all.sum()

1.0

## Save the Trained Model

In [122]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

**Challenge** Create a full matrix from the sparse_test_data. Time the function call. How long does it take? Separate the features and the target values. Save these as separate .txt files: a TEST_TARGET_FILE and a TEST_FEATURE_MARIX file. 


In [124]:
# Creates a full matrix from the sparse_test_data. 
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

C:\Users\tanic\AppData\Local\Temp\ipykernel_22676\3471363982.py:20: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_matrix.fillna(value=0, inplace=True)


In [125]:
# Separates the features and the target values. 
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY'] # features
y_test = full_test_data.CATEGORY # targets

In [126]:
# Saves the features and targets as .txt files:
# TEST_TARGET_FILE and TEST_FEATURE_MARIX 
np.savetxt(TEST_FEATURE_MATRIX, X_test)
np.savetxt(TEST_TARGET_FILE, y_test)